# Lab 1: Problem 1 (MDP algorithms: tabular Q-learning)

*OpenAI gym FrozenLake environment*

Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following

        SFFF
        FHFH
        FFFH
        HFFG

    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located

    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.
    
    FrozenLake-v0 defines "solving" as getting average reward of 0.78 over 100 consecutive trials.


In [156]:
#wandb set up for logging runs online and moving them to the leaderboard
import wandb
wandb.login()
run=wandb.init(project="lab1", tags=["problem1"], entity="ieor-4575")

wandb: wandb version 0.10.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [157]:
## DO NOT CHANGE THIS CELL
import numpy as np
import gym
env=gym.make('FrozenLake-v0')
env.seed(0)

[0]

For proper accounting rewards while you learn, we build a wrapper around env.step() and env.reset(). In an episode, every time you take an action the reward will be appended to the reward of the episode, and when ever the environment is reset (at the end of an epsiode), the episode reward is reset to 0. 

In [158]:
## DO NOT CHANGE THIS CELL
#wrapper for accounting rewards
rEpisode=0
rList=[]
fixedWindow=100
movingAverage=0

def reset_decorate(func):
    def func_wrapper():
        global rList
        global movingAverage
        global rEpisode
        global fixedwindow
        rList.append(rEpisode)
        if len(rList) >= fixedWindow:
            movingAverage=np.mean(rList[len(rList)-fixedWindow:len(rList)-1])
        rEpisode=0
        return(func())
    return func_wrapper

env.reset = reset_decorate(env.reset)

def step_decorate(func):
    def func_wrapper(action):
        global rEpisode
        s1, r, d, other = func(action)
        rEpisode+=r
        return(s1, r, d, other)
    return func_wrapper

env.step = step_decorate(env.step)

def init():
    rEpisode=0
    rList=[]
    movingAverage=0
    return

Below we illustrate the execution of the Open AI gym enviornment using the policy of chosing random action in every state. Every time an action is taken the enviorment returns a tuple containing next state, reward, and the status (whether terminal state is reached or not). 

In [159]:
### RANDOM SAMPLING EXAMPLE
num_episodes=3000; #number of episodes you want to try
episode_max_length=100; #you can explicitly end the epsiode before terminal state is reached

env.reset()
#env.render()
#execute in episodes
for i in range(num_episodes):
    
    #reset the environment at the beginning of an episode
    s = env.reset()
    d = False #not done
    
    for t in range(episode_max_length):
        
        ################ Random action policy ###########################
        #play random action 
        a = env.action_space.sample()
        #get new state, reward, done
        s, r, d, _ = env.step(a)
        #################################################################
        
        
        #break if done, reached terminal state 
        if d == True:
            break
    
    
    #log per-episode reward and moving average over 100 episodes
    wandb.log({ "random reward" : rEpisode, "random reward moving average" : movingAverage, "random episode" : i})

Implement tabular Q-learning (*YOU SHOULD ONLY CHANGE THE CELL BELOW*)

In [160]:
import random 
#initialize episodic structure
init()
num_episodes=20000; #number of training episodes, you can increase this to train more
episode_max_length=100;

np.random.seed(256) # random stay same for tunning param 

#initialize discount factor, learning rate
gamma=0.9
# Best performance when gamma = 0.95

learnRate=0.63

# 0.7 --> 0.667
# 0.65 --> 0.7677 
# 0.64 --> 0.434  
# 0.63 --> 0.7778 
# 0.625 --> 0.5859 
# 0.6 --> 0.727  



#create Q table
Q=np.zeros([env.observation_space.n, env.action_space.n]) #Q(s,a). The Q-values from this array will be used to evaluate your policy.
n=np.ones([env.observation_space.n, env.action_space.n])  #recording number of trails for each arm
n_actions = env.action_space.n
rewards_history = [] 

explor_rate = 1.0
min_explor_rate = 0.001  
max_explor_rate = 1.0
explor_decay_rate = 0.005 # 0.005 is reasonable from experiment 

#execute in episodes
for i in range(num_episodes):
    #reset the environment at the beginning of an episode
    s = env.reset()
    d = False #not done
    episode_rewards = 0
    for j in range(episode_max_length):
        
        ###########SELCT ACTION a for state s using Q-values ##################
        #example
        #a = np.argmax(Q[s,:])
        #a = env.action_space.sample()
         
        random_num = random.uniform(0, 1) 
        if random_num <= explor_rate:
            a = env.action_space.sample()
        else:
            a = np.argmax(Q[s,:]) 
         
        #get new state, reward, done
        s1, r, d,_ = env.step(a)
        
        #update Q(s,a)
        Q[s,a] = (1 - learnRate) * Q[s,a] + learnRate * (r + gamma*np.max(Q[s1,:]))
        episode_rewards += r 
    
        #break if done, reached terminal state 
        if d == True:
            break
        s=s1
        
    explor_rate = min_explor_rate + \
    (max_explor_rate - min_explor_rate) * np.exp(-explor_decay_rate*(i))
    
    rewards_history.append(episode_rewards)
    #log per-episode reward and moving average over 100 episodes
    wandb.log({ "training reward" : rEpisode, "training reward moving average" : movingAverage, "training episode" : i})
wandb.run.summary["number of training episodes"]=num_episodes

rewards_per_thousand_episodes = np.split(np.array(rewards_history),num_episodes/1000) 
count = 1000 
print(" --------- Avg reward per 1000 episodes --------- \n")
for r in rewards_per_thousand_episodes:
    print("Iter ", count, ": ", str(sum(r/1000)))
    count += 1000

 --------- Avg reward per 1000 episodes --------- 

Iter  1000 :  0.17200000000000013
Iter  2000 :  0.4930000000000004
Iter  3000 :  0.6250000000000004
Iter  4000 :  0.5880000000000004
Iter  5000 :  0.6590000000000005
Iter  6000 :  0.6010000000000004
Iter  7000 :  0.6510000000000005
Iter  8000 :  0.6150000000000004
Iter  9000 :  0.6360000000000005
Iter  10000 :  0.5900000000000004
Iter  11000 :  0.6430000000000005
Iter  12000 :  0.6650000000000005
Iter  13000 :  0.6690000000000005
Iter  14000 :  0.5810000000000004
Iter  15000 :  0.6020000000000004
Iter  16000 :  0.5850000000000004
Iter  17000 :  0.5720000000000004
Iter  18000 :  0.5570000000000004
Iter  19000 :  0.6180000000000004
Iter  20000 :  0.5870000000000004


In [161]:
%%wandb
## DO NOT CHANGE THIS CELL. CHANGING ANY PART OF THIS CELL CAN DISQUALIFY THE SUBMISSION
#Evaluation of trained policy
init()
num_episodes=1000; #number of episodes for evaluation
episode_max_length=100; 
movingAverageArray=[]
score=0
env.reset()
for i in range(num_episodes):
    s = env.reset()
    d = False #not done
    for t in range(episode_max_length):
        a = np.argmax(Q[s,:])
        s, r, d, _ = env.step(a)
        if d == True:
            break
    #log per-episode reward and moving average over 100 episodes
    wandb.log({ "evaluation reward" : rEpisode, "evaluation reward moving average" : movingAverage, "evaluation episode" : i})
    movingAverageArray.append(movingAverage)
    #score is x if there is a window of 100 consecutive episodes where moving average was at least x
    if i>100:
        score=max(score,min(movingAverageArray[i-100:i-1]))


wandb.run.summary["score"]=score 

In [162]:
run.finish()

random reward,0.0
random reward moving average,0.0101
random episode,2999
_runtime,31
_timestamp,1614302123
_step,23999
training reward,1.0
training reward moving average,0.73737
training episode,19999
number of training episodes,20000
evaluation reward,1.0


random reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
random reward moving average,▁▁▃▅▃▃▅▃▅██▅▅▁▃▅▃▆▆▃▅██▅▁▁▃▅▃▃▅▅▁▁▁▃▅▃▃▃
random episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
_timestamp,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
training reward,▁▁████▁█▁█▁▁███▁███████████▁███▁█▁█████▁
training reward moving average,▁▂▄▆▅▇▆▄▇▇▆▇▇▇▇▇▇▆▄▇▆▆█▇▇▆▆▄▅▇▅▅▇▆▆▅▇█▇█
training episode,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
evaluation reward,▁█████████▁▁██▁█▁██▁▁▁█▁███▁▁█▁▁███▁████
evaluation reward moving average,▅▅▅▇▇▇▆▆▇███▆▅▅▅▅▅▆▅█▇▄▃▁▃▃▄▅▂▄▅▄▆▅▅▃▃▃▄
